# StegaShield No-ML Prototype

Run this notebook in Google Colab after unzipping the project and uploading your images.


In [ ]:
# Optional: Mount Google Drive (if your project/images are stored there)
# Run this cell if you need to access files from Google Drive

import os

try:
    from google.colab import drive
    
    mountpoint = '/content/drive'
    
    # Check if already mounted
    is_mounted = os.path.ismount(mountpoint) if os.path.exists(mountpoint) else False
    
    if is_mounted:
        print("✅ Google Drive is already mounted!")
    else:
        # Check if mountpoint has files that need to be cleared
        if os.path.exists(mountpoint) and os.listdir(mountpoint):
            print("⚠️ Mountpoint has files. Attempting to unmount first...")
            print("💡 If unmount fails, you may need to restart the runtime")
            # Try unmounting using shell command (works better in Colab)
            os.system('fusermount -u /content/drive 2>/dev/null || umount /content/drive 2>/dev/null || true')
        
        # Now try to mount
        try:
            drive.mount(mountpoint, force_remount=False)
            print("✅ Google Drive mounted successfully!")
        except ValueError as e:
            error_msg = str(e).lower()
            if "already contain files" in error_msg:
                print("⚠️ Mountpoint still has files after unmount attempt.")
                print("💡 Solution: Restart the runtime (Runtime → Restart runtime)")
                print("   Then run this cell again, or skip it if Drive is already accessible.")
            elif "already mounted" in error_msg:
                print("✅ Google Drive appears to be mounted already!")
                print("💡 You can access it at /content/drive")
            else:
                print(f"⚠️ Mount error: {e}")
                print("💡 Try restarting the runtime and running this cell again")
        except Exception as mount_error:
            print(f"⚠️ Could not mount: {mount_error}")
            print("💡 You can skip this if your files are uploaded directly to Colab")
            
except ImportError:
    print("ℹ️ Not running in Colab, skipping Google Drive mount")
except Exception as e:
    print(f"⚠️ Error: {e}")
    print("💡 You can skip this if your files are uploaded directly to Colab")


In [ ]:
!pip install pywavelets opencv-python pillow matplotlib --quiet


In [ ]:
from pathlib import Path
import csv
import sys
import os

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Try to detect the project directory automatically
# Common Colab paths - adjust if your project is in a different location
possible_paths = [
    Path("/content/stegashield_noml_full"),
    Path("/content/stegashield_model"),
    Path("/content/drive/MyDrive/stegashield_model"),  # Google Drive
    Path("/content/drive/MyDrive/stegashield_noml_full"),  # Google Drive
    Path("/content"),
    Path.cwd(),  # Current working directory
]

PROJECT_DIR = None
for path in possible_paths:
    if (path / "models" / "hybrid_multidomain_embed_det.py").exists():
        PROJECT_DIR = path
        break

if PROJECT_DIR is None:
    # If not found, use current directory or let user specify
    PROJECT_DIR = Path.cwd()
    print(f"⚠️ Could not auto-detect project directory. Using: {PROJECT_DIR}")
    print("💡 If imports fail, set PROJECT_DIR manually to where you uploaded the project.")
else:
    print(f"✅ Found project at: {PROJECT_DIR}")

IMAGE_DIR = Path("/content/images")  # folder with your originals
OUTPUT_DIR = PROJECT_DIR / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Add the project directory to Python path (parent directory so 'models' package can be found)
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

# Verify the models directory exists
models_path = PROJECT_DIR / "models" / "hybrid_multidomain_embed_det.py"
if not models_path.exists():
    print(f"❌ Error: Cannot find models at {models_path}")
    print(f"📁 Current directory: {Path.cwd()}")
    print(f"📁 Files in PROJECT_DIR: {list(PROJECT_DIR.iterdir()) if PROJECT_DIR.exists() else 'Directory does not exist'}")
else:
    print(f"✅ Models found at: {models_path.parent}")

from models.hybrid_multidomain_embed_det import HybridMultiDomainEmbedderDet
from models.hybrid_multidomain_verify_det import HybridMultiDomainVerifierDet
from models.semi_fragile_dwt_svd import (
    SemiFragileEmbedderDwtSvd,
    SemiFragileVerifierDwtSvd,
    DwtSvdParams,
)
from training.test_harness_det import TestHarness
from utils.visualization import compute_heatmap, side_by_side

print("✅ All imports successful!")


In [ ]:
LSB_MESSAGE = "StegaShield_Dataset2025"
SEMI_MESSAGE = "StegaShield_SemiFragile"
MAX_IMAGES = 50  # change to ~1000 if you like

image_paths = []
for ext in ("*.png", "*.jpg", "*.jpeg", "*.bmp"):
    image_paths.extend(sorted(IMAGE_DIR.glob(ext)))

print(f"Found {len(image_paths)} images in {IMAGE_DIR}")
if not image_paths:
    print("⚠️ Please upload some images to IMAGE_DIR.")


In [ ]:
# Demo: single-image watermark + heatmap
if image_paths:
    sample_path = image_paths[0]
    print("Sample:", sample_path)
    embedder = HybridMultiDomainEmbedderDet()
    meta = embedder.embed(
        image_path=str(sample_path),
        message=LSB_MESSAGE,
        output_path=str(OUTPUT_DIR / (sample_path.stem + "_lsb_sample.png")),
        metadata_path=str(OUTPUT_DIR / (sample_path.stem + "_lsb_sample_metadata.json")),
    )
    orig = Image.open(sample_path).convert("RGB")
    wm = Image.open(meta["image_path"]).convert("RGB")
    heat = compute_heatmap(orig, wm)
    combined = side_by_side(orig, heat)
    
    # Display with matplotlib to ensure proper handling of non-square images
    plt.figure(figsize=(12, 6))
    plt.imshow(combined)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("No images to demo.")


In [ ]:
# Run full harness on a batch
if image_paths:
    harness = TestHarness(output_dir=str(OUTPUT_DIR), lsb_message=LSB_MESSAGE, semi_message=SEMI_MESSAGE)
    subset = [str(p) for p in image_paths[:MAX_IMAGES]]
    results_csv = harness.run_batch(subset, csv_name="results.csv")
    print("Results written to:", results_csv)
else:
    print("No images for batch testing.")


In [ ]:
# Plot aggregated robustness metrics
csv_path = OUTPUT_DIR / "results.csv"
if csv_path.exists():
    import collections

    bit_acc = collections.defaultdict(list)
    fragile = collections.defaultdict(list)

    with csv_path.open("r", newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            attack = row["attack"]
            layer = row["layer"]
            if layer == "SemiFragile":
                try:
                    ba = float(row["bit_accuracy"]) if row["bit_accuracy"] not in ("", "None") else 0.0
                except ValueError:
                    ba = 0.0
                bit_acc[attack].append(ba)
            elif layer == "LSB":
                # Handle case-insensitive fragile_match values (True/TRUE/False/FALSE)
                fragile_match_str = str(row["fragile_match"]).strip().upper()
                fragile_match = fragile_match_str == "TRUE"
                fragile[attack].append(1.0 if fragile_match else 0.0)

    attacks = sorted(set(list(bit_acc.keys()) + list(fragile.keys())))
    avg_bit = [sum(bit_acc[a]) / len(bit_acc[a]) if bit_acc[a] else 0.0 for a in attacks]
    avg_fragile = [sum(fragile[a]) / len(fragile[a]) if fragile[a] else 0.0 for a in attacks]

    # Plot Semi-Fragile robustness
    plt.figure(figsize=(14, 5))
    plt.bar(range(len(attacks)), avg_bit, color='steelblue', alpha=0.7)
    plt.xticks(range(len(attacks)), attacks, rotation=45, ha="right")
    plt.ylabel("Avg Bit Accuracy (SemiFragile)")
    plt.title("Semi-Fragile DWT–SVD Robustness vs Attack")
    plt.ylim(0, max(avg_bit) * 1.1 if max(avg_bit) > 0 else 1.0)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot LSB Fragile Detection
    plt.figure(figsize=(14, 5))
    bars = plt.bar(range(len(attacks)), avg_fragile, color='crimson', alpha=0.7)
    plt.xticks(range(len(attacks)), attacks, rotation=45, ha="right")
    plt.ylabel("Fragile Hash Match Rate (LSB)")
    plt.title("Fragile Detection vs Attack (LSB Layer)")
    plt.ylim(0, 1.1)
    plt.grid(axis='y', alpha=0.3)
    # Add value labels on bars if they're visible
    for i, (bar, val) in enumerate(zip(bars, avg_fragile)):
        if val > 0.01:  # Only label if value is significant
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{val:.2f}', ha='center', va='bottom', fontsize=8)
    plt.tight_layout()
    plt.show()
else:
    print("No results.csv yet.")
